# 63-细化处理

细化是将线条宽度设置为1的过程，按照下面的算法进行处理：

1. 从左上角开始进行光栅扫描；
2. 如果$x_0(x,y)=0$，不处理。如果$x_0(x,y)=1$，满足下面三个条件时，令$x_0=0$：

            4-近邻像素的取值有一个以上为0；
            
            x_0的4-连接数为1；
            
            x0的8-近邻中有三个以上取值为1。
            
3. 重复光栅扫描，直到步骤2中像素值改变次数为$0$。
4. 用于细化的算法有Hilditch算法（问题六十四），Zhang-Suen算法（问题六十五），田村算法等。

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
img = cv2.imread('../picture/gazo.png').astype(np.float32)

In [5]:
def thining(img):
    
    H, W, C = img.shape

    out = np.zeros((H, W), dtype=np.int)
    out[img[..., 0] > 0] = 1

    count = 1
    while count > 0:
        count = 0
        tmp = out.copy()

        for y in range(H):
            for x in range(W):
                if out[y, x] < 1:
                    continue
                
                # 满足条件
                judge = 0
                
                #  条件 1
                if (tmp[y, min(x+1, W-1)] + tmp[max(y-1, 0), x] + tmp[y, max(x-1, 0)] + tmp[min(y+1, H-1), x]) < 4:
                    judge += 1
                    
                # 条件 2
                c = 0
                c += (tmp[y,min(x+1, W-1)] - tmp[y, min(x+1, W-1)] * tmp[max(y-1, 0),min(x+1, W-1)] * tmp[max(y-1, 0), x])
                c += (tmp[max(y-1,0), x] - tmp[max(y-1,0), x] * tmp[max(y-1, 0), max(x-1, 0)] * tmp[y, max(x-1, 0)])
                c += (tmp[y, max(x-1, 0)] - tmp[y,max(x-1, 0)] * tmp[min(y+1, H-1), max(x-1, 0)] * tmp[min(y+1, H-1), x])
                c += (tmp[min(y+1, H-1), x] - tmp[min(y+1, H-1), x] * tmp[min(y+1, H-1), min(x+1, W-1)] * tmp[y, min(x+1, W-1)])
                if c == 1:
                    judge += 1
                    
                # 条件 3
                if np.sum(tmp[max(y-1, 0) : min(y+2, H), max(x-1, 0) : min(x+2, W)]) >= 4:
                    judge += 1
                
                # 三个条件均满足
                if judge == 3:
                    out[y, x] = 0
                    count += 1

    out = out.astype(np.uint8) * 255

    return out

In [7]:
out = thining(img)

In [8]:
cv2.imwrite("../picture/result63_thining.jpg", out)
cv2.imshow("result", out)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 64-Hilditch 细化算法

算法如下：

1. 从左上角开始进行光栅扫描；
2. $x_0(x,y)=0$的话、不进行处理。$x_0(x,y)=1$的话，下面五个条件都满足的时候令$x_0=-1$：

        当前像素的4-近邻中有一个以上0；
        x_0的8-连接数为1；
        x_1至x_8的绝对值之和大于2；
        8-近邻像素的取值有一个以上为1；
        对所有x_n(n in [1,8])以下任一项成立：x_n不是-1；当x_n为0时，x_0的8-连接数为1。
        
3. 将每个像素的$-1$更改为$0$；
4. 重复进行光栅扫描，直到某一次光栅扫描中步骤3的变化数变为$0$。

In [11]:
def Hildithch_thining(img):
    
    H, W, C = img.shape

    out = np.zeros((H, W), dtype=np.int)
    out[img[..., 0] > 0] = 1

    count = 1
    while count > 0:
        count = 0
        tmp = out.copy()

        for y in range(H):
            for x in range(W):
                if out[y, x] < 1:
                    continue
                
                # 满足条件
                judge = 0
                
                #  条件 1
                if (tmp[y, min(x+1, W-1)] + tmp[max(y-1, 0), x] + tmp[y, max(x-1, 0)] + tmp[min(y+1, H-1), x]) < 4:
                    judge += 1
                    
                # 条件 2
                _tmp = tmp
                _tmp = 1 - _tmp
                c = 0
                c += (_tmp[y,min(x+1, W-1)] - _tmp[y, min(x+1, W-1)] * _tmp[max(y-1, 0),min(x+1, W-1)] * _tmp[max(y-1, 0), x])
                c += (_tmp[max(y-1,0), x] - _tmp[max(y-1,0), x] * _tmp[max(y-1, 0), max(x-1, 0)] * _tmp[y, max(x-1, 0)])
                c += (_tmp[y, max(x-1, 0)] - _tmp[y,max(x-1, 0)] * _tmp[min(y+1, H-1), max(x-1, 0)] * _tmp[min(y+1, H-1), x])
                c += (_tmp[min(y+1, H-1), x] - _tmp[min(y+1, H-1), x] * _tmp[min(y+1, H-1), min(x+1, W-1)] * _tmp[y, min(x+1, W-1)])
                if c == 1:
                    judge += 1
                    
                # 条件 3
                s = np.abs(tmp[y-1, x]) + np.abs(tmp[y-1, x+1]) + np.abs(tmp[y, x-1]) + np.abs(tmp[y, x]) + np.abs(tmp[y, x+1]) + np.abs(tmp[y+1, x-1]) + np.abs(tmp[y+1, x]) + np.abs(tmp[y+1, x+1])
                if s > 2:
                    judge += 1
                    
                # 条件 4
                if np.sum(tmp[max(y-1, 0) : min(y+2, H), max(x-1, 0) : min(x+2, W)]) >= 2:
                    judge += 1
                    
                # 条件 5
                if out[y, x] != -1:
                    continue
                    if out[y, x] == 0:
                        judge +=1
                
                # 三个条件均满足
                if judge == 5:
                    out[y, x] = 0
                    count += 1

    out = out.astype(np.uint8) * 255

    return out

In [12]:
out = Hildithch_thining(img)

In [13]:
cv2.imwrite("../picture/result64_thining.jpg", out)
cv2.imshow("result", out)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 65-Zhang-Suen细化算法

请注意，有必要反转gazo.png的值，因为以下所有操作都将0作为线，将1作为背景。

对于中心像素$x_1(x,y)$的$8-$近邻定义如下： $$ \begin{matrix} x_9&x_2&x_3\ x_8&x_1&x_4\ x_7&x_6&x_5 \end{matrix} $$ 考虑以下两个步骤：

步骤一：执行光栅扫描并标记满足以下5个条件的所有像素：

这是一个黑色像素；

顺时针查看$x_2$、$x_3$、$\cdots$、$x_9$、$x_2$时，从$0$到$1$​的变化次数仅为$1$；

$x_2$、$x_3$、$\cdots$、$x_9$中$1$的个数在$2$个以上$6$个以下；

$x_2$、$x_4$、$x_6$中的一个为1；

$x_4$、$x_6$、$x_8$中的一个为1；

将标记的像素全部变为$1$。

步骤二：执行光栅扫描并标记满足以下5个条件的所有像素：

这是一个黑色像素；
顺时针查看$x_2$、$x_3$、$\cdots$、$x_9$、$x_2$时，从0到1的变化次数仅为1；
$x_2$、$x_3$、$\cdots$、$x_9$中$1$的个数在$2$个以上$6$个以下；
$x_2$、$x_4$、$x_6$中的一个为1；
$x_2$、$x_6$、$x_8$中的一个为1；
将标记的像素全部变为$1$。

反复执行步骤一和步骤二直到没有点变化

In [14]:
def Zhang_Suen_thining(img):
    
    H, W, C = img.shape

    out = np.zeros((H, W), dtype=np.int)
    out[img[..., 0] > 0] = 1

    # 反转
    out = 1 - out

    while True:
        s1 = []
        s2 = []

        # step 1 ( rasta scan )
        for y in range(1, H-1):
            for x in range(1, W-1):
                
                # 条件 1
                if out[y, x] > 0:
                    continue

                # 条件 2
                f1 = 0
                if (out[y-1, x+1] - out[y-1, x]) == 1:
                    f1 += 1
                if (out[y, x+1] - out[y-1, x+1]) == 1:
                    f1 += 1
                if (out[y+1, x+1] - out[y, x+1]) == 1:
                    f1 += 1
                if (out[y+1, x] - out[y+1,x+1]) == 1:
                    f1 += 1
                if (out[y+1, x-1] - out[y+1, x]) == 1:
                    f1 += 1
                if (out[y, x-1] - out[y+1, x-1]) == 1:
                    f1 += 1
                if (out[y-1, x-1] - out[y, x-1]) == 1:
                    f1 += 1
                if (out[y-1, x] - out[y-1, x-1]) == 1:
                    f1 += 1

                if f1 != 1:
                    continue
                    
                # 条件 3
                f2 = np.sum(out[y-1:y+2, x-1:x+2])
                if f2 < 2 or f2 > 6:
                    continue
                
                # 条件 4
                if out[y-1, x] + out[y, x+1] + out[y+1, x] < 1:
                    continue

                # 条件 5
                if out[y, x+1] + out[y+1, x] + out[y, x-1] < 1:
                    continue
                    
                s1.append([y, x])

        for v in s1:
            out[v[0], v[1]] = 1

        # step 2 ( rasta scan )
        for y in range(1, H-1):
            for x in range(1, W-1):
                
                # 条件 1
                if out[y, x] > 0:
                    continue

                # 条件 2
                f1 = 0
                if (out[y-1, x+1] - out[y-1, x]) == 1:
                    f1 += 1
                if (out[y, x+1] - out[y-1, x+1]) == 1:
                    f1 += 1
                if (out[y+1, x+1] - out[y, x+1]) == 1:
                    f1 += 1
                if (out[y+1, x] - out[y+1,x+1]) == 1:
                    f1 += 1
                if (out[y+1, x-1] - out[y+1, x]) == 1:
                    f1 += 1
                if (out[y, x-1] - out[y+1, x-1]) == 1:
                    f1 += 1
                if (out[y-1, x-1] - out[y, x-1]) == 1:
                    f1 += 1
                if (out[y-1, x] - out[y-1, x-1]) == 1:
                    f1 += 1

                if f1 != 1:
                    continue
                    
                # 条件 3
                f2 = np.sum(out[y-1:y+2, x-1:x+2])
                if f2 < 2 or f2 > 6:
                    continue
                
                # 条件 4
                if out[y-1, x] + out[y, x+1] + out[y, x-1] < 1:
                    continue

                # 条件 5
                if out[y-1, x] + out[y+1, x] + out[y, x-1] < 1:
                    continue
                    
                s2.append([y, x])

        for v in s2:
            out[v[0], v[1]] = 1

        # 没有像素变化
        if len(s1) < 1 and len(s2) < 1:
            break

    out = 1 - out
    out = out.astype(np.uint8) * 255

    return out

In [15]:
out = Zhang_Suen_thining(img)